In [1]:
from enum import Enum, auto
import tensorflow as tf
from tensorflow.keras import activations, Sequential, layers
from tensorflow_addons.layers import SpectralNormalization

In [2]:
image_shape = (128, 128, 3)
n_encoder_channels = 16
n_decoder_channels = 16

mult = 1
scale_factor = 2

latent_dim = 128
res_cells_per_group = 1
n_groups_per_scale = [2] * 6
n_latent_scales = len(n_groups_per_scale)

In [3]:
# class RescaleType(Enum):
#     UP = auto()
#     DOWN = auto()


# class SqueezeExcitation(tf.keras.Model):
#     """Squeeze and Excitation block as defined by Hu, et al. (2019)
#     See Also
#     ========
#     Source paper https://arxiv.org/pdf/1709.01507.pdf
#     """
#     def __init__(self, ratio=16, **kwargs) -> None:
#         super().__init__(**kwargs)
#         self.ratio = ratio

#     def build(self, input_shape):
#         batch_size, h, w, c = input_shape
#         self.gap = layers.GlobalAveragePooling2D(data_format="channels_last")
#         num_hidden = max(c / self.ratio, 4)
#         self.dense1 = layers.Dense(units=num_hidden)
#         self.dense2 = layers.Dense(units=c)
    
#     def call(self, inputs):
#         x = self.gap(inputs)
#         x = self.dense1(x)
#         x = activations.relu(x)
#         x = self.dense2(x)
#         x = activations.sigmoid(x)
#         x = tf.expand_dims(x, 1)
#         x = tf.expand_dims(x, 2)
#         return x * inputs

In [4]:
# class EncodingResidualCell(tf.keras.Model):
#     """Encoding network residual cell in NVAE architecture"""
#     def __init__(self, output_channels, **kwargs):
#         super().__init__(**kwargs)
#         self.batch_norm1 = layers.BatchNormalization(momentum=0.05, epsilon=1e-5)
#         self.conv1 = SpectralNormalization(layers.Conv2D(output_channels, (3, 3), padding="same"))
#         self.batch_norm2 = layers.BatchNormalization(momentum=0.05, epsilon=1e-5)
#         self.conv2 = SpectralNormalization(layers.Conv2D(output_channels, (3, 3), padding="same"))
#         self.se = SqueezeExcitation()

#     def call(self, inputs):
#         x = activations.swish(self.batch_norm1(inputs))
#         x = self.conv1(x)
#         x = activations.swish(self.batch_norm2(x))
#         x = self.conv2(x)
#         x = self.se(x)
#         return 0.1 * inputs + x

In [5]:
# class GenerativeResidualCell(tf.keras.Model):
#     """Generative network residual cell in NVAE architecture"""
#     def __init__(self, output_channels, expansion_ratio=6, **kwargs):
#         super().__init__(**kwargs)
        
#         self.batch_norm1 = layers.BatchNormalization(momentum=0.05, epsilon=1e-5)
#         self.conv1 = SpectralNormalization(layers.Conv2D(expansion_ratio * output_channels, (1, 1), padding="same"))
#         self.batch_norm2 = layers.BatchNormalization(momentum=0.05, epsilon=1e-5)
#         self.depth_conv = layers.DepthwiseConv2D((5, 5), padding="same")
#         self.batch_norm3 = layers.BatchNormalization(momentum=0.05, epsilon=1e-5)
#         self.conv2 = SpectralNormalization(layers.Conv2D(output_channels, (1, 1), padding="same"))
#         self.batch_norm4 = layers.BatchNormalization(momentum=0.05, epsilon=1e-5)
#         self.se = SqueezeExcitation()

#     def call(self, inputs):
#         x = self.batch_norm1(inputs)
#         x = self.conv1(x)
#         x = activations.swish(self.batch_norm2(x))
#         x = self.depth_conv(x)
#         x = activations.swish(self.batch_norm3(x))
#         x = self.conv2(x)
#         x = self.batch_norm4(x)
#         x = self.se(x)
#         return 0.1 * inputs + x

In [6]:
# class Rescaler(tf.keras.Model):
#     def __init__(self, n_channels, scale_factor, rescale_type, **kwargs) -> None:
#         super().__init__(**kwargs)
        
#         self.bn = layers.BatchNormalization(momentum=0.05, epsilon=1e-5)
#         self.mode = rescale_type
#         self.factor = scale_factor
        
#         if rescale_type == RescaleType.UP:
#             self.conv = SpectralNormalization(layers.Conv2D(n_channels, (3, 3), strides=(1, 1), padding="same"))
            
#         elif rescale_type == RescaleType.DOWN:
#             self.conv = SpectralNormalization(layers.Conv2D(n_channels, (3, 3), strides=(self.factor, self.factor), padding="same"))

#     def call(self, input):
#         x = self.bn(input)
#         x = activations.swish(x)
        
#         if self.mode == RescaleType.UP:
#             _, height, width, _ = x.get_shape()
#             x = tf.image.resize(x, size=(self.factor * height, self.factor * width), method="nearest")
#         x = self.conv(x)
#         return x

In [7]:
# def create_encoder_layers(n_latent_scales, n_groups_per_scale, res_cells_per_group, n_encoder_channels, scale_factor, mult=1):
#     # create encoder layers as a list
#     enc_layers = []
#     for scale in range(n_latent_scales):
#         n_groups = n_groups_per_scale[scale]
#         print('\nGroup: ', scale)

#         for group_idx in range(n_groups):
#             if scale == group_idx == 0:
#                 output_channels = n_encoder_channels * mult
#             print('Output_channels: ', output_channels)
            
#             for rb in range(res_cells_per_group):
#                 enc_layers.append(EncodingResidualCell(output_channels, name='res_block_' + str(scale) + '_' + str(group_idx) + '_' + str(rb)))
#                 print('res block')
        
#         # We downsample in the end of each scale except last
#         if scale < n_latent_scales - 1:
#             output_channels = n_encoder_channels * mult * scale_factor
#             enc_layers.append(Rescaler(output_channels, scale_factor=scale_factor, rescale_type=RescaleType.DOWN))
#             print('Rescaler')
#             print('New output_channels: ', output_channels)
#             mult *= scale_factor
    
#     enc_layers.append(layers.ELU())
#     enc_layers.append(SpectralNormalization(layers.Conv2D(n_encoder_channels * mult, (1, 1), padding="same")))
#     enc_layers.append(layers.ELU())
    
#     return enc_layers, mult

In [8]:
# def create_decoder_layers(encoder_mult, n_latent_scales, n_groups_per_scale, res_cells_per_group, n_decoder_channels, scale_factor):
#     m = encoder_mult
#     dec_layers = []
    
#     for scale in range(n_latent_scales):
#         print('\nGroup: ', scale)
#         n_groups = n_groups_per_scale[scale]

#         for group in range(n_groups):
#             if scale == group == 0:
#                 output_channels = int(n_decoder_channels * m)
#             print('Output channels', output_channels)
            
#             for res in range(res_cells_per_group):
#                 dec_layers.append(GenerativeResidualCell(output_channels))
#                 print('Gen Res block', flush=True)

#         if scale < n_latent_scales - 1:
#             output_channels = int(n_decoder_channels * m / scale_factor)
            
#             dec_layers.append(Rescaler(output_channels, scale_factor=scale_factor, rescale_type=RescaleType.UP))
#             print('Rescaler', flush=True)

#             m /= scale_factor
#     return dec_layers

In [9]:
# class Sampling(layers.Layer):
#     def call(self, inputs):
#         z_mean, z_log_var = inputs
#         batch = tf.shape(z_mean)[0]
#         dim = tf.shape(z_mean)[1]
#         epsilon = tf.random.normal([batch, dim])
#         return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [10]:
# x_in = layers.Input(shape=image_shape, name='encoder_input')
# x = SpectralNormalization(layers.Conv2D(n_encoder_channels, (3, 3), padding="same"))(x_in)

# enc_layers, mult = create_encoder_layers(n_latent_scales, n_groups_per_scale, res_cells_per_group, n_encoder_channels, scale_factor)

# for group in enc_layers:
#     x = group(x)
    
# before_shape = x.shape[1:]
# x = layers.GlobalAveragePooling2D()(x)
# after_shape = x.shape[1:][0]

# z_mean = layers.Dense(latent_dim, name='z_mean')(x)
# z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)
# z = Sampling()([z_mean, z_log_var])

# model_encoder = tf.keras.models.Model(x_in, [z_mean, z_log_var, z], name='encoder')
# model_encoder.summary()

# print(before_shape, after_shape)

In [11]:
# decoder_input = layers.Input(shape=(latent_dim), name='decoder_input')
# x = layers.Dense(before_shape[0] * before_shape[1] * before_shape[2])(decoder_input)
# x = layers.Reshape(before_shape)(x)

# dec_layers = create_decoder_layers(mult, n_latent_scales, n_groups_per_scale, res_cells_per_group, n_decoder_channels, scale_factor)

# for group in dec_layers:
#     x = group(x)
    
# x = layers.Activation(activations.elu)(x)
# x = SpectralNormalization(layers.Conv2D(3, kernel_size=(3, 3), padding="same"))(x)

# model_decoder = tf.keras.models.Model(decoder_input, x, name='decoder')
# model_decoder.summary()

In [12]:
def squeeze_excite_block(input_x, ratio=16):
    shp = getshape_layer()(input_x)
    batch_size, h, w, c = shp
    num_hidden = max(c / ratio, 4)
    x = layers.GlobalAveragePooling2D()(input_x)
    x = layers.Dense(units=num_hidden, activation='relu')(x)
    x = layers.Dense(units=c, activation='sigmoid')(x)
    x = layers.Reshape((1, 1, c))(x)
    x = layers.Multiply()([input_x, x])
    return x

def encoding_res_block(input_x, output_channels):
    x = layers.BatchNormalization(momentum=0.05, epsilon=1e-5)(input_x)
    x = layers.Activation(activations.swish)(x)
    x = layers.Conv2D(output_channels, (3, 3), padding="same")(x)
    
    x = layers.BatchNormalization(momentum=0.05, epsilon=1e-5)(x)
    x = layers.Activation(activations.swish)(x)
    x = layers.Conv2D(output_channels, (3, 3), padding="same")(x)
    x = squeeze_excite_block(x)
    
    x = layers.Add()([0.1 * input_x, x])
    return x

def generative_res_block(input_x, output_channels, expansion_ratio=6):
    x = layers.BatchNormalization(momentum=0.05, epsilon=1e-5)(input_x)
    x = layers.Conv2D(expansion_ratio * output_channels, (1, 1), padding="same")(x)
    x = layers.BatchNormalization(momentum=0.05, epsilon=1e-5)(x)
    x = layers.Activation(activations.swish)(x)
    x = layers.DepthwiseConv2D((5, 5), padding="same")(x)
    x = layers.BatchNormalization(momentum=0.05, epsilon=1e-5)(x)
    x = layers.Activation(activations.swish)(x)
    x = layers.Conv2D(output_channels, (1, 1), padding="same")(x)
    x = layers.BatchNormalization(momentum=0.05, epsilon=1e-5)(x)
    x = squeeze_excite_block(x)
    
    x = layers.Add()([0.1 * input_x, x])
    return x
    
def rescaler(input_x, n_channels, scale_factor, rescale_type):
    x = layers.BatchNormalization(momentum=0.05, epsilon=1e-5)(input_x)
    x = layers.Activation(activations.swish)(x)
    
    if rescale_type == 'up':
        shp = getshape_layer()(x)
        _, height, width, _ = shp
        x = tf.image.resize(x, size=(scale_factor * height, scale_factor * width), method="nearest")
        x = layers.Conv2D(n_channels, (3, 3), strides=(1, 1), padding="same")(x)
    else:
        x = layers.Conv2D(n_channels, (3, 3), strides=(scale_factor, scale_factor), padding="same")(x)
    return x

class getshape_layer(layers.Layer):
    def call(self, inputs):
        return inputs.shape

class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.random.normal([batch, dim])
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

#########################################################

def create_encoder(mult=1):
    x_in = layers.Input(shape=image_shape, name='encoder_input')
    x = layers.Conv2D(n_encoder_channels, (3, 3), padding="same")(x_in)
    
    for scale in range(n_latent_scales):
        n_groups = n_groups_per_scale[scale]
        print('\nGroup: ', scale)

        for group_idx in range(n_groups):
            output_channels = n_encoder_channels * mult
            print('Output_channels: ', output_channels)
            
            for rb in range(res_cells_per_group):
                x = encoding_res_block(x, output_channels)
                print('res block')
        
        # We downsample in the end of each scale except last
        if scale < n_latent_scales - 1:
            output_channels = n_encoder_channels * mult * scale_factor
            x = rescaler(x, output_channels, scale_factor, 'down')
            print('Rescaler')
            print('New output_channels: ', output_channels)
            mult *= scale_factor
    
    x = layers.ELU()(x)
    x = layers.Conv2D(n_encoder_channels * mult, (1, 1), padding="same")(x)
    x = layers.ELU()(x)
    
    before_shape = x.shape[1:]
    x = layers.GlobalAveragePooling2D()(x)
    
    z_mean = layers.Dense(latent_dim, name='z_mean')(x)
    z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)
    z = Sampling()([z_mean, z_log_var])

    model_encoder = tf.keras.models.Model(x_in, [z_mean, z_log_var, z], name='encoder')
    model_encoder.summary()
    return model_encoder, before_shape, mult

#########################################################

def create_decoder(en_mult, before_shape):
    decoder_input = layers.Input(shape=(latent_dim), name='decoder_input')
    x = layers.Dense(before_shape[0] * before_shape[1] * before_shape[2])(decoder_input)
    x = layers.Reshape(before_shape)(x)
    
    for scale in range(n_latent_scales):
        print('\nGroup: ', scale)
        n_groups = n_groups_per_scale[scale]

        for group in range(n_groups):
            output_channels = int(n_decoder_channels * en_mult)
            print('Output channels', output_channels)
            
            for res in range(res_cells_per_group):
                x = generative_res_block(x, output_channels, expansion_ratio=6)
                print('Gen Res block', flush=True)

        if scale < n_latent_scales - 1:
            output_channels = int(n_decoder_channels * en_mult / scale_factor)
            x = rescaler(x, output_channels, scale_factor, 'up')
            print('Rescaler', flush=True)
            en_mult /= scale_factor
    
    x = layers.Conv2D(3, kernel_size=(3, 3), padding="same")(x)
    x = layers.Activation(activations.sigmoid)(x)
    
    model_decoder = tf.keras.models.Model(decoder_input, x, name='decoder')
    model_decoder.summary()
    return model_decoder

In [13]:
model_encoder, before_shape, en_mult = create_encoder()
model_decoder = create_decoder(en_mult, before_shape)


Group:  0
Output_channels:  16


AttributeError: 'TensorShape' object has no attribute 'shape'